In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

# OpenAI Whisper API를 활용한 음성-텍스트 변환 튜토리얼 (2025년 3월 기준)


OpenAI의 Whisper 모델은 사람의 음성을 높은 정확도로 텍스트로 변환하는 자동 음성 인식(ASR) 모델입니다.</br> 2025년 현재 Whisper는 OpenAI API를 통해 제공되며,</br> 음성 파일을 원어 그대로 <strong style="color:red">텍스트로 필사(transcription)</strong>하거나 영어로 <strong style="color:red">번역(translation)</strong>하는 기능을 제공합니다.</br> 이 튜토리얼에서는 Whisper API를 사용하여 음성-텍스트 변환을 구현하는 방법을 단계별로 살펴보겠습니다.</br> 환경 설정부터 기본 사용 방법, 그리고 언어 감지, 타임스탬프 포함 출력, 실시간 음성 처리와 같은 고급 기능까지 다룹니다.

## 1. 환경 설정

Python 코드에서 python-dotenv로 .env 파일을 불러온 뒤, openai.OpenAI() 클래스를 이용해 API 클라이언트 인스턴스를 생성합니다.</br> 이때 API 키는 명시적으로 전달하거나 환경 변수 OPENAI_API_KEY가 설정되어 있어야 합니다.</br> 환경 변수 또는 인자로 API 키를 지정하지 않으면 OpenAI 라이브러리는 다음과 같은 오류를 발생시킵니다.</br></br>

> OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

아래는 환경 로드 및 클라이언트 생성 예제입니다.


In [2]:
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()
client = OpenAI()

위 코드에서는 .env에서 불러온 API 키로 client 객체를 생성했습니다.</br> 이제 이 client를 통해 DALL-E를 비롯한 OpenAI API 요청을 보낼 수 있습니다.</br> (참고로, api_key를 생략하면 OPENAI_API_KEY 환경 변수를 자동으로 참조합니다.)

## 2. Whisper API 소개 및 기본 음성 → 텍스트 변환

OpenAI Whisper API는 두 가지 주요 <strong style="color:red;">엔드포인트(endpoint)</strong>를 제공합니다.</br>
- Transcriptions 엔드포인트: 제공한 음성 파일을 해당 음성의 원어로 필사(글로 변환) 합니다 </br>(예: 한국어 음성을 입력하면 한국어 텍스트로 출력).</br></br>
- Translations 엔드포인트: 제공한 음성 파일을 영어로 번역하여 텍스트로 반환합니다.</br></br>

Whisper API는 현재 Whisper v2 대형 모델(whisper-1로 식별)을 사용하며, MP3, MP4, WAV 등 다양한 음성 파일 형식을 지원합니다.</br>
다만 한 번 요청할 수 있는 파일 크기는 최대 25MB로 제한되어 있으므로 긴 오디오의 경우 분할하여 처리해야 합니다.</br></br>

우선 MP3 파일 등의 정적 파일을 Whisper로 <strong style="color:red;">텍스트 변환(필사)</strong>하는 기본 방법을 살펴보겠습니다. 예제로 간단한 영어 음성 MP3 파일을 텍스트로 변환해보겠습니다.</br>

### 기본 사용법: 음성 파일 필사 요청
1. 오디오 파일 준비: 변환하고자 하는 음성 파일의 경로를 지정합니다.</br> Whisper API는 여러 형식의 오디오를 지원하는데, 일반적으로 wav나 mp3를 많이 사용합니다</br>. 여기서는 예시로 speech.mp3 파일을 사용하겠습니다.</br></br>
2. API 요청 구성: OpenAI Python SDK의 client.audio.transcriptions.create 메서드를 사용하여 음성 필사 요청을 보냅니다.</br> 주요 파라미터:
    <p>- file: 열어둔 오디오 파일 객체 ("rb" 모드로 연 파일),
    - model: 사용할 모델 ID ("whisper-1"로 지정하여 Whisper 모델 사용).</br>
    기본적으로 Whisper API는 응답으로 JSON 형식 데이터를 반환하며, 그 안에 "text" 필드로 변환된 텍스트를 제공합니다.</p></br> response_format 파라미터를 사용해 응답 형식을 변경할 수 있는데, 기본값 "json" 외에 "text", "srt" 등 다양한 옵션이 있습니다.</br> 우선 이해를 돕기 위해 응답을 순수 텍스트로 받도록 response_format="text"를 지정해보겠습니다.</br></br>

3. 응답 처리: Whisper API는 음성 내용을 텍스트로 변환하여 반환합니다.</br> response_format="text"로 요청한 경우 변환된 텍스트 문자열을 직접 반환하므로, 이를 출력하거나 변수에 저장할 수 있습니다.</br></br>

다음은 MP3 파일을 읽어서 Whisper로 전송하고, 결과 텍스트를 출력하는 기본 코드 예제입니다


In [4]:
# 1. 음성 파일 열기
audio_file = open('data/speech.mp3','rb')

# 2. whisper API에 음성 필사 요청
response = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    response_format="text", # 텍스트만 반환
    )
audio_file.close()
# 3. 변환된 텍스트 출력 
print(response)


OpenAI Whisper is an advanced speech recognition model.



In [6]:
from IPython.display import Audio
Audio(filename="data/speech.mp3", autoplay=True)

위 코드가 실행되면, speech.mp3 파일의 음성이 텍스트로 변환되어 response에 담기고, 그 내용을 출력합니다.</br> 예를 들어 입력 음성이 영어로 "OpenAI Whisper is an advanced speech recognition model." 라는 문장을 담고 있었다면, </br>출력은 다음과 같은 텍스트가 될 것입니다.</br>
```
OpenAI Whisper is an advanced speech recognition model.
```
이제 Whisper API의 기본 동작을 확인했으므로, 추가적인 기능들과 상황별 활용법을 알아보겠습니다.</br>

## 3. 다양한 언어의 음성 인식 및 언어감지

Whisper 모델은 다국어 음성 인식을 지원합니다.</br> 90개가 넘는 언어로 훈련되어 다양한 언어의 음성을 텍스트로 변환할 수 있습니다.</br> 기본적으로 API에 언어를 명시하지 않으면 Whisper가 음성의 언어를 자동으로 감지하여 해당 언어로 텍스트를 필사합니다.</br> 예를 들어, 한국어 음성을 입력하면 자동으로 한국어로 텍스트를 출력합니다.</br></br> 

언어 감지 및 지정 (language 파라미터):</br>

자동 감지가 대부분 잘 동작하지만, 경우에 따라 잘못된 언어로 인식될 가능성도 있습니다.</br> Whisper API는 이런 경우를 대비해 language 파라미터로 입력 음성의 언어를 직접 지정하는 기능을 제공합니다.</br> language 파라미터에 ISO 언어 코드를 설정하면 모델이 해당 언어로 인식하도록 강제할 수 있습니다.</br>
- 예를 들어, 한국어 음성 파일을 확실히 한국어로 인식시키고 싶다면 language="ko"로 지정할 수 있습니다.</br></br>
- 영어라면 language="en", 스페인어라면 language="es" 등으로 언어 코드를 설정합니다.</br>

다음은 스페인어 음성 파일을 Whisper로 변환하는 코드 예시입니다.</br> Whisper가 자동으로 언어를 감지하게 할 수도 있지만,</br> 예시에서는 language="es"를 명시적으로 지정해보겠습니다.

In [7]:
# 스페인어 필사
with open('data/spanish_audio.mp3','rb') as audio_file:
    response_es = client.audio.transcriptions.create(
        file=audio_file,
        model='whisper-1',
        response_format='text',
        language='es', # 입력 음성 언어를 지정(스페인어)
        )
print(response_es)

¿Qué crees que es la inteligencia artificial?



In [8]:
# 한국어 필사
with open('data/output_ko_onyx.mp3','rb') as audio_file:
    transcript_ko = client.audio.transcriptions.create(
        file = audio_file,
        model='whisper-1',
        response_format='text',
        language='ko')             # 입력 음성 언어  지정
print(transcript_ko)

안녕하세요. 저는 OpenAPI로 생성된 한국어 음성입니다. 한국어로 자동인식되어 읽고 있습니다.



만약 spanish_audio.mp3에 스페인어로 "¿Qué crees que es la inteligencia artificial?" 라는 문장이 담겨 있었다면, 위 코드의 출력은 다음과 같이 스페인어 텍스트가 될 것입니다.</br>
```
¿Qué crees que es la inteligencia artificial?
```

Whisper에 언어를 지정함으로써 모델이 해당 언어로 정확히 인식하도록 도울 수 있습니다.</br> 대부분의 경우 언어를 지정하지 않아도 잘 동작하지만, 언어 감지 결과를 신뢰하기 어렵거나 특정 언어로 출력되길 원한다면 language 파라미터를 활용하세요.</br></br>

## 4. 고급 기능 1: 타임스탬프 포함 자막 및 세부 출력

기본적인 Whisper API 응답은 전체 음성을 하나의 텍스트로 반환하지만,</br> 경우에 따라 문장별로 시간 구간을 표시하거나 단어별 타임스탬프가 필요한 경우도 있습니다 </br>(예: 동영상 자막 생성, 정교한 음성 분석 등).</br> Whisper API는 이러한 요구에 맞게 여러 가지 출력 형식을 제공합니다.

### 다양한 출력 형식 (response_format 옵션)
response_format 파라미터를 조정하여 Whisper의 출력 형식을 바꿀 수 있습니다:

- "json" (기본값): { "text": "..." } 형태의 JSON 응답. 추가 메타정보 없이 최종 텍스트만 포함합니다.</br></br></br></br>
- "text": 순수 텍스트로 변환 결과만 반환. (이미 앞서 사용)</br></br>
- "srt": SubRip 자막 형식으로 반환. 자막처럼 각 문장에 시간 코드가 포함됩니다.</br></br>
- "vtt": WebVTT 자막 형식으로 반환. (srt와 유사한 자막 표준 형식)</br></br>
- "verbose_json": 인식 결과에 대한 상세 정보를 담은 JSON으로 반환. 인식된 문장별 세그먼트와 타임스탬프, 확률 등 메타데이터가 모두 포함됩니다.</br></br> 자막 형식 출력: 예를 들어 음성 파일을 response_format="srt"으로 요청하면, Whisper는 출력 텍스트를 일정 길이로 나누어 각 부분에 시작-끝 시간이 포함된 자막 형식으로 돌려줍니다.</br></br> 

다음 코드를 통해 이전과 동일한 speech.mp3에 대해 SRT 형식 출력을 받아보겠습니다


In [9]:
# 1. 음성 파일 열기
audio_file = open('data/speech.mp3','rb')

# 2. whisper API에 음성 필사 요청
response = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    response_format="srt", # 텍스트만 반환
    )
audio_file.close()
# 3. 변환된 텍스트 출력 
print(response)

1
00:00:00,000 --> 00:00:04,000
OpenAI Whisper is an advanced speech recognition model.





응답은 문자열 형태로 SRT 콘텐츠가 반환됩니다.</br></br>

위 예시처럼, SRT 포맷에서는 각 블록마다 순번, 시작/끝 시간, 그리고 해당 구간의 대사가 나타납니다.</br> Whisper API가 자동으로 음성을 몇 초 단위로 구분해 자막으로 만들어준 것을 확인할 수 있습니다.</br></br>

세부 JSON 출력: 더 구조적인 정보가 필요하다면 response_format="verbose_json"을 사용할 수 있습니다.</br> 이 모드에서는 음성 전체를 몇 개의 **세그먼트(segment)**로 나누고, 각 세그먼트에 대한 텍스트, 시작/종료 시간 정보(timestamp), 신뢰도 등의 정보를 JSON 형식으로 제공합니다.</br> 이를 활용하면 필요에 따라 세그먼트를 직접 처리하거나 자막을 커스터마이징할 수 있습니다.


In [10]:
with open('data/ch5_shimmer(한국어).wav','rb') as audio_file:
    transcript_json = client.audio.transcriptions.create(
    file=audio_file,
    model='whisper-1',
    response_format="verbose_json",
    language='ko'
    )
transcript_json

TranscriptionVerbose(duration=7.800000190734863, language='korean', text='안녕하세요. 저는 OpenAPI로 생성된 한국어 음성입니다. 한국어로 자동인식되어 읽고 있어요.', segments=[TranscriptionSegment(id=0, avg_logprob=-0.3058373034000397, compression_ratio=1.0521739721298218, end=5.099999904632568, no_speech_prob=0.009894744493067265, seek=0, start=0.0, temperature=0.0, text=' 안녕하세요. 저는 OpenAPI로 생성된 한국어 음성입니다.', tokens=[50364, 19289, 13, 10551, 7238, 4715, 40, 12888, 6439, 8631, 27930, 21045, 3103, 15179, 8631, 7416, 13, 50619]), TranscriptionSegment(id=1, avg_logprob=-0.3058373034000397, compression_ratio=1.0521739721298218, end=7.800000190734863, no_speech_prob=0.009894744493067265, seek=0, start=5.099999904632568, temperature=0.0, text=' 한국어로 자동인식되어 읽고 있어요.', tokens=[50619, 21045, 6540, 1955, 15905, 8309, 4215, 10436, 22826, 3103, 43302, 1313, 12654, 13, 50754])], usage=Usage(duration=None, type='duration', seconds=8), words=None, task='transcribe')

In [18]:
segment = transcript_json.segments[0]
# print('text 시작 시간:', segments.start)
# print('text 끝나는 시간 :', segments.end)
print("text구간 {}=>{}".format(segment.start, segment.end))
print(segment.text)

text구간 0.0=>5.099999904632568
 안녕하세요. 저는 OpenAPI로 생성된 한국어 음성입니다.


위 코드에서 transcription_json은 segments에 여러 세그먼트가 리스트로 포함됩니다.</br> 각 세그먼트에는 "start", "end" (초 단위 시간), "text" 등이 담겨 있으며, 이 외에도 "confidence"(평균 예측 확률)나 "language"(감지된 언어) 등의 부가 정보가 있을 수 있습니다.</br>

## 5. 고급 기능 2: 실시간 음성 변환 (마이크 입력 처리)

지금까지는 파일에 저장된 음성을 변환하는 방법을 다루었습니다.</br> 이번에는 실시간 마이크 입력을 받아서 Whisper API로 전송하는 방법을 살펴보겠습니다.</br> 실시간 처리란 말 그대로 사용자가 말하는 동시에(text) 바로바로 텍스트로 변환하는 것을 의미합니다.</br> Whisper API는 스트리밍 엔드포인트를 제공하지는 않으나, 짧은 구간으로 녹음하여 연속으로 API에 보내는 방식으로 유사 실시간 처리가 가능합니다.</br>
>주의: 네트워크 API를 이용하는 이상 완벽한 실시간(동시에 변환) 처리에는 한계가 있습니다.</br> 음성을 일정 간격으로 끊어서 전송하면 약간의 지연은 발생하지만, 빠른 응답 속도를 감안하면 거의 실시간에 가까운 자막을 구현할 수 있습니다.</br> 진정한 실시간 처리가 필요하다면 오픈소스 Whisper 모델을 로컬에서 사용하거나 스트리밍 기능을 지원하는 서비스를 검토해야 합니다.</br>

### 마이크 입력 녹음하여 전송하기
Python에서는 sounddevice나 pyaudio 등의 라이브러리를 사용하여 마이크로부터 오디오 데이터를 받을 수 있습니다.</br> 여기서는 sounddevice를 활용한 간단한 예제를 소개합니다.</br></br>

1. 짧은 음성 녹음: </br>sounddevice.rec 함수를 사용하면 지정한 초 만큼 마이크로부터 오디오를 녹음할 수 있습니다. 아래 예시는 5초간 모노 음성을 16kHz로 녹음합니다.</br></br>
2. 오디오 데이터를 파일로 저장:</br> Whisper API에 전송하려면 오디오 데이터를 파일 형태 (또는 파일 객체)로 제공해야 합니다.</br> scipy.io.wavfile.write 함수를 이용해 녹음한 데이터를 WAV 파일로 저장합니다.</br></br>
3. 저장한 파일 전송:</br> 앞서와 동일하게 client.audio.transcriptions.create를 사용하여 Whisper API로 WAV 파일을 보내면 됩니다.</br></br>

코드 예시는 다음과 같습니다 (5초간 녹음 후 즉시 전송).


In [22]:
import sounddevice as sd
from scipy.io.wavfile import write  # mp3는 스트리밍이 안 되어, wav로 스트리밍

fs = 16000 # 샘플레이트 16kHz
seconds = 5 # 음성녹음 길이 5초
print('지금부터 5초간 녹음합니다.')

recording = sd.rec(int(seconds*fs), samplerate=fs, channels=1, dtype="int16")
sd.wait()
file_name = 'data/ch6_live_input.wav'
write(file_name, fs, recording)
print('녹음된 음성이 저장되었습니다.(크기:{}byte)'.format(len(recording)))

# from IPython.display import Audio
# Audio(filename=file_name, autoplay=True)

# 저장된 오디오(마이크에 입려된 데이터)를 whisper API로 전송하여 text로 받기
with open(file_name, 'rb') as audio_file:
    live_transcript = client.audio.transcriptions.create(
        file = audio_file,
        model= 'whisper-1',
        response_format='text',
        language='ko'
    )
print('실시간 녹음 변환된 text :', live_transcript)

# 파일 삭제 
import os
try:
    os.remove(file_name)
    print(f"{file_name} 파일이 삭제되었습니다.")
except Exception as e:
          print(f'파일 삭제 중 오류 발생 : {e}')

지금부터 5초간 녹음합니다.
녹음된 음성이 저장되었습니다.(크기:80000byte)
실시간 녹음 변환된 text : ... ... ...

data/ch6_live_input.wav 파일이 삭제되었습니다.


# <strong style="color:red;">6. Quiz </strong>
1. 과제 개요
교육생들은 OpenAI의 최신 API를 활용하여 주어진 텍스트 파일을 읽고, 내용을 요약한 후, 이를 음성 파일(MP3)로 변환하는 파이썬 프로그램을 개발해야 합니다.
본 과제를 통해 학생들은 다음과 같은 핵심 기술을 학습할 수 있습니다.

- OpenAI API를 활용한 텍스트 요약
- OpenAI의 텍스트 음성 변환(TTS) 기능 적용
- Python 환경에서 파일 입출력 처리
- python-dotenv를 활용한 API 키 관리

2. 요구 사항
```
a. 텍스트 파일 읽기: 프로그램은 지정된 텍스트 파일(ch06_quiz.txt)을 읽어와야 합니다.
b. 텍스트 요약: OpenAI API를 사용하여 긴 텍스트를 70자 이내로 요약해야 합니다.
c. 음성 변환: 요약된 내용을 OpenAI의 TTS(Text-to-Speech) API를 활용하여 음성데이터로 변환합니다.
d. MP3 파일 저장: 생성된 음성 데이터를 ch06_quiz.mp3 파일로 저장해야 합니다.
e. 환경변수 사용: API 키는 코드에 직접 포함하지 않고, .env 파일을 이용해 관리해야 합니다.
f. 지정된 보이스 사용: OpenAI TTS에서 제공하는 특정 보이스("nova")를 사용해야 합니다.


In [45]:
from dotenv import load_dotenv
import openai
load_dotenv()
client = openai.OpenAI()

In [24]:
with open('data/ch06_quiz.txt', encoding='utf-8') as f:
    quiz_text = f.readline()
quiz_text = quiz_text.replace('\n','').replace('\t','').strip()
print(quiz_text)

인공지능(AI)은 인간이나 동물의 자연 지능과 달리 기계가 보여주는 지능을 의미합니다. 주요 AI 교재에서는 이를 "지능형 에이전트"의 연구로 정의하며, 이는 환경을 인식하고 목표를 달성하기 위해 행동하는 시스템을 의미합니다.


In [25]:
response = client.chat.completions.create(
    model='gpt-4.1-nano',
    messages=[
        {'role':'system','content':'글자를 70자 이내로 요약해줘'},
        {'role':'user','content':quiz_text}
        
    ])

In [30]:
response

ChatCompletion(id='chatcmpl-BmyLVytseavCOe6azOBe2ZZzWYENz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='AI는 기계의 지능으로, 환경 인식 및 목표 달성 행동을 하는 시스템입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1751011413, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_38343a2f8f', usage=CompletionUsage(completion_tokens=22, prompt_tokens=91, total_tokens=113, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [37]:
first_quiz_text = response.choices[0].message.content
first_quiz_text

'AI는 기계의 지능으로, 환경 인식 및 목표 달성 행동을 하는 시스템입니다.'

In [36]:
type(first_quiz_text)

str

In [42]:
quiz_sound = client.audio.speech.create(
    model='tts-1',
    voice='nova',
    input=first_quiz_text,
    speed=0.9)
audio_quiz_data = quiz_sound.content
with open('data/ch6_quiz.mp3', 'wb') as audio_quiz:
    audio_quiz.write(audio_quiz_data)